In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
import joblib
import os
from datetime import datetime, timedelta

In [11]:
# Importa o custom transformer
import sys
# Adiciona a raiz do projeto ao PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Adicionado '{project_root}' ao sys.path para importações.")
else:
    print(f"'{project_root}' já está no sys.path.")
from src.utils.custom_transformers import DateFeatureEngineer
print("DateFeatureEngineer importado com sucesso!")

'c:\Users\murilo.weber\projetos_portfolio\tcrm_opportunity_loss_prediction' já está no sys.path.
DateFeatureEngineer importado com sucesso!


# --- 0. Carregamento do DataFrame Consolidado ---


In [12]:
# Define o caminho para a pasta 'processed'
processed_data_path = '../data/processed/'
file_path = os.path.join(processed_data_path, 'df_eda_consolidated.csv')

try:
    df = pd.read_csv(file_path)
    print("DataFrame carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{file_path}' não encontrado.")
    print("Certifique-se de que o notebook '01_data_understanding.ipynb' foi executado e salvou 'df_eda_consolidated.csv'.")
    exit() # Interrompe a execução se o arquivo não for encontrado


DataFrame carregado com sucesso!


In [13]:
print("\n--- Primeiras 5 linhas do DataFrame ---")
df.head()


--- Primeiras 5 linhas do DataFrame ---


,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,target,sector,year_established,revenue,employees,office_location,subsidiary_of,series,sales_price,manager,regional_office
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Engaging,2016-10-20,2017-03-01,1054.0,0,retail,2001.0,718.62,2448.0,United States,NaN,GTX,1096.0,Dustin Brinkmann,Central
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Prospecting,2016-10-25,2017-03-11,4514.0,0,medical,2002.0,3178.24,4540.0,United States,NaN,NaN,NaN,Melvin Marxen,Central
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Engaging,2016-10-25,2017-03-07,50.0,0,retail,2001.0,718.62,2448.0,United States,NaN,MG,55.0,Melvin Marxen,Central
3,MV1LWRNH,Moses Frase,GTX Basic,Codehow,Engaging,2016-10-25,2017-03-09,588.0,0,software,1998.0,2714.90,2641.0,United States,Acme Corporation,GTX,550.0,Dustin Brinkmann,Central
4,PE84CX4O,Zane Levy,GTX Basic,Hatfan,Engaging,2016-10-25,2017-03-02,517.0,0,services,1982.0,792.46,1299.0,United States,NaN,GTX,550.0,Summer Sewald,West


In [5]:
print("\n--- Informações do DataFrame ---")
print(df.info())
print("\n--- Fim do carregamento ---")


--- Informações do DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   opportunity_id    8800 non-null   object 
 1   sales_agent       8800 non-null   object 
 2   product           8800 non-null   object 
 3   account           7375 non-null   object 
 4   deal_stage        8800 non-null   object 
 5   engage_date       8300 non-null   object 
 6   close_date        6711 non-null   object 
 7   close_value       7487 non-null   float64
 8   target            8800 non-null   int64  
 9   sector            7375 non-null   object 
 10  year_established  7375 non-null   float64
 11  revenue           7375 non-null   float64
 12  employees         7375 non-null   float64
 13  office_location   7375 non-null   object 
 14  subsidiary_of     1292 non-null   object 
 15  series            7320 non-null   object 
 16  sales_pr

# --- 1. Preparação para o Pipeline: Definição de Features ---

In [6]:
# A coluna 'opportunity_duration_days' será criada PELO Custom Transformer.

# As colunas que não são 'opportunity_id', 'account', 'deal_stage' e 'target'
# serão as entradas para o X_preprocessor_input.
# As colunas 'engage_date' e 'close_date' serão processadas pelo DateFeatureEngineer.
# 'deal_stage' será removido pois não é uma feature para o modelo.

columns_to_drop_for_X_before_pipeline = ['opportunity_id', 'account', 'deal_stage'] # Apenas as colunas que não são features para o modelo
existing_cols_to_drop = [col for col in columns_to_drop_for_X_before_pipeline if col in df.columns]
df_for_pipeline_input = df.drop(columns=existing_cols_to_drop, errors='ignore')

print(f"\n--- Colunas removidas ANTES do pipeline para formar X_preprocessor_input: {existing_cols_to_drop} ---")
print(f"Número de colunas após remoção: {df_for_pipeline_input.shape[1]}")


--- Colunas removidas ANTES do pipeline para formar X_preprocessor_input: ['opportunity_id', 'account', 'deal_stage'] ---
Número de colunas após remoção: 16


In [7]:
# Salvar df_processed_for_modeling.csv (ainda é útil para análise fora do pipeline)
# A única diferença é que as colunas de data ainda estarão presentes neste CSV,
# mas o pipeline principal as removerá internamente.
processed_data_output_path = '../data/processed/'
if not os.path.exists(processed_data_output_path):
    os.makedirs(processed_data_output_path)
    print(f"Diretório '{processed_data_output_path}' criado.")

df_final_filename = os.path.join(processed_data_output_path, 'df_processed_for_modeling.csv')
# Salva o DF com as datas, pois o pipeline as receberá.
df_for_pipeline_input.to_csv(df_final_filename, index=False)
print(f"\n--- DataFrame final processado (com colunas de data para o pipeline) salvo em: '{df_final_filename}' ---")


--- DataFrame final processado (com colunas de data para o pipeline) salvo em: '../data/processed/df_processed_for_modeling.csv' ---


# --- 2. Construção do Pipeline de Pré-processamento ---

In [8]:
# Definindo X e y
X_preprocessor_input = df_for_pipeline_input.drop('target', axis=1)
y_target = df_for_pipeline_input['target']

print(f"Dimensões de X (input para o pre-processador): {X_preprocessor_input.shape}")
print(f"Dimensões de y: {y_target.shape}")

Dimensões de X (input para o pre-processador): (8800, 15)
Dimensões de y: (8800,)


In [9]:
print(X_preprocessor_input.columns.tolist())

['sales_agent', 'product', 'engage_date', 'close_date', 'close_value', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of', 'series', 'sales_price', 'manager', 'regional_office']


## 2.1. Definição das colunas para cada tipo de transformação

In [9]:
numeric_features = [
    'close_value',
    'year_established',
    'revenue',
    'employees',
    'sales_price'
]

In [10]:
categorical_features = [
    'sales_agent',
    'product',
    'sector',
    'office_location',
    'subsidiary_of',
    'series',
    'manager',
    'regional_office'
]

In [11]:
date_features = ['engage_date', 'close_date']

In [12]:
# Verificação de que as features existem em X_preprocessor_input
missing_numeric = [f for f in numeric_features if f not in X_preprocessor_input.columns]
missing_categorical = [f for f in categorical_features if f not in X_preprocessor_input.columns]
missing_date = [f for f in date_features if f not in X_preprocessor_input.columns]

if missing_numeric or missing_categorical or missing_date:
    raise ValueError(f"Features missing from X_preprocessor_input: Numeric: {missing_numeric}, Categorical: {missing_categorical}, Date: {missing_date}")

print(f"\n--- Features numéricas para escalonamento: {numeric_features} ---")
print(f"--- Features categóricas para One-Hot Encoding: {categorical_features} ---")
print(f"--- Features de data para engenharia: {date_features} ---")



--- Features numéricas para escalonamento: ['close_value', 'year_established', 'revenue', 'employees', 'sales_price'] ---
--- Features categóricas para One-Hot Encoding: ['sales_agent', 'product', 'sector', 'office_location', 'subsidiary_of', 'series', 'manager', 'regional_office'] ---
--- Features de data para engenharia: ['engage_date', 'close_date'] ---


## 2.2. Criação do Pré-processador (ColumnTransformer)

In [13]:
# Este ColumnTransformer atuará SOMENTE sobre as colunas que estarão presentes
# DEPOIS que o DateFeatureEngineer já tiver feito seu trabalho.
# Ou seja, 'opportunity_duration_days' estará presente, e 'engage_date'/'close_date' não.

# ATENÇÃO: A lista 'numeric_features' PRECISA conter 'opportunity_duration_days'
# APÓS o DateFeatureEngineer ter atuado.
# No fit do preprocessor, o DateFeatureEngineer vai criar essa coluna.
# Então, o ColumnTransformer espera a 'opportunity_duration_days' na lista de features numéricas.

numeric_features_with_duration = [
    'close_value',
    'year_established',
    'revenue',
    'employees',
    'sales_price',
    'opportunity_duration_days' # Adiciona a feature que será criada
]

preprocessor_steps = ColumnTransformer(
    transformers=[
        ('num_pipeline', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features_with_duration), # Aplica a cols numéricas (incluindo a futura duração)

        ('cat_pipeline', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
        ]), categorical_features) # Aplica a cols categóricas
    ],
    remainder='passthrough'
)

In [14]:
# Agora, o pipeline completo será: DateFeatureEngineer -> preprocessor_steps (ColumnTransformer)
# e este será o 'preprocessor' final que será salvo.
# Note que `X_preprocessor_input` ainda conterá as colunas de data originais.
# O pipeline final tratará isso.

# Criando o Pipeline completo de pré-processamento
# (Primeiro o DateFeatureEngineer, depois o ColumnTransformer)
preprocessor = Pipeline(steps=[
    ('date_feature_engineer', DateFeatureEngineer()),
    ('column_transformer_steps', preprocessor_steps)
])

## 2.3. Treinar o pre-processador (ColumnTransformer)

In [15]:
# Ele precisa ser ajustado nos dados de treino para aprender as escalas e categorias.
# Para isso, usaremos o X_preprocessor_input completo para ajustá-lo.
print("\n--- Ajustando o Pipeline de Pré-processamento (ColumnTransformer) ---")
preprocessor.fit(X_preprocessor_input)

print("--- Pipeline de Pré-processamento ajustado com sucesso! ---")


--- Ajustando o Pipeline de Pré-processamento (ColumnTransformer) ---
--- Pipeline de Pré-processamento ajustado com sucesso! ---


# --- 3. Salvamento do Pipeline de Pré-processamento ---

In [16]:
models_dir = '../models/'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Diretório '{models_dir}' criado com sucesso.")

# Salvar APENAS o pipeline de pré-processamento
preprocessor_pipeline_filename = os.path.join(models_dir, 'preprocessor_pipeline.joblib')
joblib.dump(preprocessor, preprocessor_pipeline_filename) # Salva o ColumnTransformer diretamente

print(f"\n--- Pipeline de Pré-processamento salvo com sucesso em: '{preprocessor_pipeline_filename}' ---")
print("--- Fim do Processamento de Dados para Pré-processador ---")


--- Pipeline de Pré-processamento salvo com sucesso em: '../models/preprocessor_pipeline.joblib' ---
--- Fim do Processamento de Dados para Pré-processador ---
